In [192]:
# ------------ Install (run if needed) -------------
# !pip install torch torchvision pillow efficientnet-pytorch scikit-learn matplotlib seaborn

# ------------ Imports & Device Setup -------------
import os
import json
import subprocess
from typing import Dict, Any

import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)


✅ Using device: cuda


In [ ]:
# ------------ Structured Knowledge Base (DICTIONARY) -------------

KNOWLEDGE = {
    "Diseases": {
        "Pepper Bell Bacterial Spot": {
            "Symptoms": "Small dark water-soaked spots on leaves and fruits, yellow halos, leaf drop.",
            "Cause": "Xanthomonas bacteria.",
            "Organic Control": "Neem oil (5 ml/L), copper spray (2 g/L).",
            "Chemical Control": "Copper oxychloride (3 g/L), streptomycin sprays.",
            "Prevention": "Disease-free seeds, crop rotation, avoid overhead watering.",
            "Fertilizer Recommendation": "Balanced NPK 10-10-10.",
            "Climate Conditions": "High humidity, warm temperature (25–30°C).",
            "Confusion With": "Magnesium deficiency, sunscald."
        },

        "Pepper Bell Healthy": {
            "Condition": "Normal deep-green leaves, strong stems.",
            "Fertilizer Recommendation": "NPK 12-12-17 every 25 days.",
            "Prevention": "Proper irrigation, balanced fertilizer."
        },

        "Potato Early Blight": {
            "Symptoms": "Brown concentric rings on lower leaves, yellowing.",
            "Cause": "Alternaria solani.",
            "Organic Control": "Neem oil (3 ml/L).",
            "Chemical Control": "Mancozeb (2.5 g/L).",
            "Prevention": "Crop rotation, avoid overhead irrigation.",
            "Fertilizer Recommendation": "NPK 5-10-20."
        },

        "Potato Late Blight": {
            "Symptoms": "Dark greasy lesions, white mold under leaves, tuber rot.",
            "Cause": "Phytophthora infestans.",
            "Organic Control": "Neem oil (4 ml/L), copper oxychloride (3 g/L).",
            "Chemical Control": "Mancozeb, Metalaxyl.",
            "Prevention": "Remove infected plants, avoid water stagnation.",
            "Climate Conditions": "Cool + humid weather."
        },

        "Potato Healthy": {
            "Condition": "Dark green leaves, strong stems.",
            "Prevention": "Adequate nitrogen, well-drained soil.",
            "Fertilizer Recommendation": "NPK 13-40-13 at planting → 20-20-20."
        },

        "Tomato Bacterial Spot": {
            "Symptoms": "Small dark leaf spots, scabby fruits.",
            "Cause": "Xanthomonas bacteria.",
            "Organic Control": "Neem oil, copper sprays.",
            "Chemical Control": "Copper fungicides.",
            "Prevention": "Avoid overhead watering.",
            "Fertilizer Recommendation": "NPK 5-10-10."
        },

        "Tomato Early Blight": {
            "Symptoms": "Concentric brown rings, leaf yellowing.",
            "Cause": "Alternaria solani.",
            "Organic Control": "Compost tea, neem oil.",
            "Chemical Control": "Mancozeb, chlorothalonil.",
            "Prevention": "Remove infected leaves.",
            "Fertilizer Recommendation": "NPK 10-20-20."
        },

        "Tomato Late Blight": {
            "Symptoms": "Greasy lesions, white mold under leaves.",
            "Cause": "Phytophthora infestans.",
            "Organic Control": "Neem, copper oxychloride.",
            "Chemical Control": "Mancozeb, Metalaxyl.",
            "Prevention": "Remove infected plants.",
            "Fertilizer Recommendation": "Increase potassium."
        },

        "Tomato Leaf Mold": {
            "Symptoms": "Yellow patches on top of leaf, olive mold underside.",
            "Cause": "Passalora fulva.",
            "Organic Control": "Pruning, ventilation, neem spray.",
            "Chemical Control": "Copper or chlorothalonil fungicides.",
            "Prevention": "Reduce humidity."
        },

        "Tomato Septoria Leaf Spot": {
            "Symptoms": "Numerous small brown spots with pale centers.",
            "Cause": "Septoria lycopersici.",
            "Organic Control": "Neem spray.",
            "Chemical Control": "Copper fungicide.",
            "Prevention": "Avoid overhead watering."
        },

        "Tomato Spider Mites": {
            "Symptoms": "Leaf bronzing, tiny webs, yellow speckles.",
            "Cause": "Mites.",
            "Organic Control": "Neem oil (2 ml/L), soap spray.",
            "Prevention": "Increase humidity."
        },

        "Tomato Target Spot": {
            "Symptoms": "Circular spots with rings.",
            "Cause": "Corynespora cassiicola.",
            "Organic Control": "Neem, sanitation.",
            "Chemical Control": "Fungicides.",
            "Prevention": "Good airflow."
        },

        "Tomato Mosaic Virus": {
            "Symptoms": "Mottled leaves, twisted shape.",
            "Cause": "Tomato mosaic virus.",
            "Organic Control": "Remove infected plants.",
            "Prevention": "Sanitize tools."
        },

        "Tomato Yellow Leaf Curl Virus": {
            "Symptoms": "Yellow curled leaves, stunting.",
            "Cause": "Whitefly-transmitted virus.",
            "Organic Control": "Sticky traps, neem oil.",
            "Prevention": "Remove infected plants."
        },

        "Tomato Healthy": {
            "Condition": "Healthy green leaves, no visible spots, good vigor.",
            "Prevention": "Proper irrigation, balanced fertilizer, good spacing.",
            "Fertilizer Recommendation": "NPK 12-12-36 during fruiting stage.",
            "Climate Conditions": "Warm, moderate humidity."
        }
    },

    "Deficiencies": {
        "Nitrogen Deficiency": {
            "Symptoms": "Pale yellow older leaves.",
            "Fix": "Compost, cow manure, urea (1 tsp/L).",
            "Prevention": "Regular compost application."
        },
        "Phosphorus Deficiency": {
            "Symptoms": "Purplish leaves, slow growth.",
            "Fix": "DAP, bone meal.",
            "Prevention": "Maintain soil pH 6–7."
        },
        "Potassium Deficiency": {
            "Symptoms": "Brown leaf edges (scorching).",
            "Fix": "MOP fertilizer.",
            "Prevention": "Balanced NPK schedule."
        },
        "Calcium Deficiency": {
            "Symptoms": "Blossom end rot.",
            "Fix": "Calcium nitrate (1 tsp/L).",
            "Prevention": "Avoid irregular watering."
        },
        "Magnesium Deficiency": {
            "Symptoms": "Yellowing between veins on older leaves.",
            "Fix": "Epsom salt spray (1 tsp/L).",
            "Prevention": "Maintain soil pH 6–7."
        },
        "Iron Deficiency": {
            "Symptoms": "Yellowing between veins on young leaves.",
            "Fix": "Ferrous sulfate spray.",
            "Prevention": "Avoid alkaline soil."
        }
    },

    "Pests": {
        "Aphids": {
            "Symptoms": "Sticky leaves (honeydew), curling.",
            "Organic Control": "Neem oil (2 ml/L), soap spray.",
            "Chemical Control": "Imidacloprid."
        },
        "Whiteflies": {
            "Symptoms": "Tiny white insects flying when disturbed.",
            "Organic Control": "Yellow sticky traps.",
            "Chemical Control": "Thiamethoxam."
        },
        "Thrips": {
            "Symptoms": "Silver streaks, leaf distortion.",
            "Control": "Spinosad, neem spray."
        },
        "Mealybugs": {
            "Symptoms": "Cotton-like clusters.",
            "Organic Control": "Spirit + soap spray."
        },
        "Fruit Worms": {
            "Symptoms": "Holes in fruits.",
            "Control": "Bt spray (Bacillus thuringiensis)."
        }
    },

    "Stages": {
        "Seedling": {"Fertilizer": "NPK 19-19-19 (half dose).", "Care": "Avoid overwatering."},
        "Vegetative": {"Fertilizer": "NPK 20-20-20.", "Care": "Watch for fungal infections."},
        "Flowering": {"Fertilizer": "NPK 15-30-15.", "Care": "Maintain calcium."},
        "Fruiting": {"Fertilizer": "NPK 12-12-17 or 14-14-21.", "Care": "Monitor pests."}
    },

    "Climate": {
        "High Humidity": {"Risk": "Leaf mold, late blight.", "Advice": "Improve ventilation."},
        "Hot Dry": {"Risk": "Spider mites.", "Advice": "Increase humidity."},
        "Cool Wet": {"Risk": "Late blight.", "Advice": "Use protective fungicide spray."}
    },

    "Dosages": {
        "Neem Oil": "2–5 ml per liter.",
        "Copper Oxychloride": "2–3 g per liter.",
        "Mancozeb": "2–2.5 g per liter.",
        "Metalaxyl": "Use as per label.",
        "Imidacloprid": "0.3 ml per liter.",
        "Epsom Salt": "1 tsp per liter.",
        "Calcium Nitrate": "1 tsp per liter."
    }
}

print("✅ Knowledge dictionary created. Diseases:", len(KNOWLEDGE["Diseases"]))


✅ Knowledge dictionary created. Diseases: 15


In [194]:
# ============================================
# CELL 3 — Offline LLM Helper (FULLY FIXED)
# ============================================

OLLAMA_MODEL = "llama3.2:1b"   # fast + mobile-friendly

def ask_offline_llm(prompt, model=OLLAMA_MODEL, timeout=60):
    """
    Windows-safe UTF-8 decoding + no background thread errors.
    Ensures Ollama responses NEVER crash the kernel.
    """
    try:
        proc = subprocess.Popen(
            ["ollama", "run", model],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=False     # <-- IMPORTANT: read raw bytes, avoid cp1252 decoding errors
        )

        # send UTF-8 prompt
        out, err = proc.communicate(prompt.encode("utf-8"), timeout=timeout)

        # decode safely
        out = out.decode("utf-8", errors="ignore")
        err = err.decode("utf-8", errors="ignore")

        if not out.strip():
            return "⚠️ No response generated by offline LLM."

        return out.strip()

    except subprocess.TimeoutExpired:
        proc.kill()
        return "❌ Ollama timeout."

    except FileNotFoundError:
        return "❌ Ollama not found. Install Ollama and add it to your PATH."

    except Exception as e:
        return f"❌ LLM error: {e}"


In [195]:
# =============================================================
# CELL 4 — OfflineAgriculturalRAG (IMPROVED + fallback logic)
# =============================================================

class OfflineAgriculturalRAG:

    def __init__(self, knowledge: Dict[str, Any]):
        self.kb = knowledge
        print("✅ RAG loaded (dict-backed)")

    # ----------------------------------------------------------
    # Retrieve disease block from dictionary (100% accurate)
    # ----------------------------------------------------------
    def get_disease_block(self, cnn_class_name, class_to_title):
        title = class_to_title.get(cnn_class_name)
        if not title:
            return {}

        block = self.kb["Diseases"].get(title, {})
        block = dict(block)
        block["name"] = title
        return block

    # ----------------------------------------------------------
    # Strict field extraction (NO hallucination allowed)
    # ----------------------------------------------------------
    def strict_extract(self, block):
        fields = [
            "Confirmed Disease", "Symptoms", "Cause", "Organic Control", "Chemical Control",
            "Fertilizer Recommendation", "Climate Conditions", "Confusion With",
            "Prevention", "Dosage Information"
        ]

        out = {}
        for f in fields:

            # Name field
            if f == "Confirmed Disease":
                out[f] = block.get("name", "Not specified in knowledge")

            # Dosage always comes from global knowledge
            elif f == "Dosage Information":
                out[f] = self.kb["Dosages"]

            else:
                out[f] = block.get(f, "Not specified in knowledge")

        return out

    # ----------------------------------------------------------
    # Natural-text fallback for chatbot use (safe summarizer)
    # ----------------------------------------------------------
    def natural_text(self, ext):
        lines = []
        add = lambda k: lines.append(f"{k}: {ext[k]}") if ext[k] != "Not specified in knowledge" else None

        add("Confirmed Disease")
        add("Symptoms")
        add("Cause")
        add("Organic Control")
        add("Chemical Control")
        add("Prevention")
        add("Fertilizer Recommendation")
        return "\n".join(lines)

    # ----------------------------------------------------------
    # Master RAG Query
    # ----------------------------------------------------------
    def query(self, user_q, prediction, class_to_title, ask_llm=False):
        block = self.get_disease_block(prediction["disease"], class_to_title)
        strict = self.strict_extract(block)
        natural = self.natural_text(strict)

        result = {
            "extraction": strict,
            "natural": natural
        }

        # ------------------------------------------------------
        # Optional LLM answer (still RAG-restricted)
        # ------------------------------------------------------
        if ask_llm:
            fertilizer_missing = (strict["Fertilizer Recommendation"] == "Not specified in knowledge")

            llm_prompt = f"""
You are an agricultural assistant. You MUST answer ONLY using this dictionary block:

<<<RAG_START>>>
{json.dumps(block, indent=2)}
<<<RAG_END>>>

User question:
{user_q}

RULES:
- No hallucination.
- Do NOT add facts not present inside the RAG block.
- If fertilizer info is missing and the user asks about fertilizer, say:
  "Fertilizer is not specified for this disease. Follow the organic or chemical control steps instead."

Answer:
"""

            result["llm_answer"] = ask_offline_llm(llm_prompt)

        return result


# instantiate RAG
rag = OfflineAgriculturalRAG(KNOWLEDGE)
print("RAG instance created.")


✅ RAG loaded (dict-backed)
RAG instance created.


In [196]:
CLASS_TO_TITLE = {
    "Pepper__bell___Bacterial_spot": "Pepper Bell Bacterial Spot",
    "Pepper__bell___healthy": "Pepper Bell Healthy",
    "Potato___Early_blight": "Potato Early Blight",
    "Potato___Late_blight": "Potato Late Blight",
    "Potato___healthy": "Potato Healthy",
    "Tomato_Bacterial_spot": "Tomato Bacterial Spot",
    "Tomato_Early_blight": "Tomato Early Blight",
    "Tomato_Late_blight": "Tomato Late Blight",
    "Tomato_Leaf_Mold": "Tomato Leaf Mold",
    "Tomato_Septoria_leaf_spot": "Tomato Septoria Leaf Spot",
    "Tomato_Spider_mites_Two_spotted_spider_mite": "Tomato Spider Mites",
    "Tomato__Target_Spot": "Tomato Target Spot",
    "Tomato__Tomato_mosaic_virus": "Tomato Mosaic Virus",
    "Tomato__Tomato_YellowLeaf__Curl_Virus": "Tomato Yellow Leaf Curl Virus",
    "Tomato_healthy": "Tomato Healthy",
}

CHECKPOINT_PATH = "checkpoints/efficientnet_b0_best.pth"
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
class_names = checkpoint["classes"]

model = EfficientNet.from_pretrained("efficientnet-b0", num_classes=len(class_names))
model.load_state_dict(checkpoint["model"])
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

print("✅ CNN loaded with", len(class_names), "classes")


Loaded pretrained weights for efficientnet-b0
✅ CNN loaded with 15 classes


In [197]:
class IntegratedPlantDiseaseSystem:
    def __init__(self, model, class_names, rag_system, class_to_title, device, transform):
        self.model = model
        self.class_names = class_names
        self.rag = rag_system
        self.class_to_title = class_to_title
        self.device = device
        self.transform = transform

    def predict_disease(self, path):
        img = Image.open(path).convert("RGB")
        x = self.transform(img).unsqueeze(0).to(self.device)
        with torch.no_grad():
            out = self.model(x)
            probs = F.softmax(out, 1)[0]
        conf, idx = torch.max(probs, 0)
        return {"disease": self.class_names[idx], "confidence": float(conf)}

    def diagnose_and_advise(self, path, question, ask_llm=False):
        pred = self.predict_disease(path)
        rag_out = self.rag.query(question, pred, self.class_to_title, ask_llm)
        return pred, rag_out

system = IntegratedPlantDiseaseSystem(model, class_names, rag, CLASS_TO_TITLE, device, transform)
print("✅ Integrated system ready")


✅ Integrated system ready


In [198]:
test_image_path = r"K:\cnndisease\00b1f292-23dd-44d4-aad3-c1ffb6a6ad5a___RS_LB 4479.JPG"
question = "Recommend fertilizer"

prediction, result = system.diagnose_and_advise(test_image_path, question, ask_llm=False)

print("🦠 Prediction:", prediction)
print("\n🔎 Strict extraction:\n", json.dumps(result["extraction"], indent=2))
print("\n📝 Natural reply:\n", result["natural"])


🦠 Prediction: {'disease': 'Potato___Late_blight', 'confidence': 0.9119618535041809}

🔎 Strict extraction:
 {
  "Confirmed Disease": "Potato Late Blight",
  "Symptoms": "Dark greasy lesions, white mold under leaves, tuber rot.",
  "Cause": "Phytophthora infestans.",
  "Organic Control": "Neem oil (4 ml/L), copper oxychloride (3 g/L).",
  "Chemical Control": "Mancozeb, Metalaxyl.",
  "Fertilizer Recommendation": "Not specified in knowledge",
  "Climate Conditions": "Cool + humid weather.",
  "Confusion With": "Not specified in knowledge",
  "Prevention": "Remove infected plants, avoid water stagnation.",
  "Dosage Information": {
    "Neem Oil": "2\u20135 ml per liter.",
    "Copper Oxychloride": "2\u20133 g per liter.",
    "Mancozeb": "2\u20132.5 g per liter.",
    "Metalaxyl": "Use as per label.",
    "Imidacloprid": "0.3 ml per liter.",
    "Epsom Salt": "1 tsp per liter.",
    "Calcium Nitrate": "1 tsp per liter."
  }
}

📝 Natural reply:
 Confirmed Disease: Potato Late Blight
Sympto

In [199]:
correct = 0
total = 0

for c in class_names:
    title = CLASS_TO_TITLE.get(c)
    block = rag.get_disease_block(c, CLASS_TO_TITLE)
    if block and block.get("name") == title:
        correct += 1
    total += 1

print("✅ Retrieval accuracy:", round((correct/total)*100, 2), "%")


✅ Retrieval accuracy: 100.0 %


In [200]:
def faithfulness_score(extraction, block):
    rag_text = json.dumps(block).lower().split()
    ext_text = json.dumps(extraction).lower().split()
    overlap = len(set(rag_text) & set(ext_text))
    return overlap / len(set(rag_text))

score = faithfulness_score(result["extraction"], rag.get_disease_block(prediction["disease"], CLASS_TO_TITLE))
print("✅ Faithfulness:", round(score*100, 2), "%")


✅ Faithfulness: 93.02 %


In [201]:
# ============================
# CELL 10 — FINAL UNIVERSAL ROUTER
# Crop-aware, no-key-error, strict fertilizer matching
# ============================

import re
import numpy as np
import json

# ---------------------------
# Helper: find disease by name tokens (crop-aware)
# ---------------------------
def find_disease_by_name_tokens(question, preferred_crop=None):
    """
    Try to find a disease whose name tokens appear in the question.
    If preferred_crop provided, prefer diseases that mention that crop.
    Returns (disease_name or None).
    """
    q = question.lower()
    # normalize disease names -> list of tokens
    candidates = []
    for dname in KNOWLEDGE["Diseases"].keys():
        d_lower = dname.lower()
        # if user mentioned crop and disease doesn't belong to that crop, deprioritize it
        crop_score = 0
        if preferred_crop and preferred_crop in d_lower:
            crop_score += 2.0
        # count how many disease tokens appear in question
        tokens = re.findall(r"[a-zA-Z]+", d_lower)
        token_hits = sum(1 for t in tokens if t and t in q)
        if token_hits > 0:
            # score: weighted by token hits and crop_score
            score = token_hits + crop_score
            candidates.append((score, dname))
    if not candidates:
        return None
    candidates.sort(reverse=True)
    return candidates[0][1]


# ----------------------------------------------------
# LEVEL 2 — Keyword Symptom Matcher (unchanged)
# ----------------------------------------------------
def match_disease_by_symptoms(question):
    q = question.lower()
    best_match = None
    best_score = 0

    for disease, info in KNOWLEDGE["Diseases"].items():
        symptoms = info.get("Symptoms", "").lower()
        words = re.findall(r"[a-zA-Z]+", symptoms)

        score = 0
        for w in words:
            if w in q:
                score += 1 + 0.3 * (len(w)/5)

        if score > best_score:
            best_match = disease
            best_score = score

    return best_match if best_score > 0 else None


# ----------------------------------------------------
# LEVEL 3 — LLM Symptom Extraction + Crop-Aware Scoring
# ----------------------------------------------------
def extract_symptoms_llm(question):
    prompt = f"""
Extract only symptom keywords.
Return JSON list.

Text: "{question}"
"""
    raw = ask_offline_llm(prompt)

    try:
        parsed = json.loads(raw)
        if isinstance(parsed, list):
            return [x.lower() for x in parsed]
    except:
        pass

    return [w for w in question.lower().split() if len(w) > 4]


def ml_style_symptom_match(question):
    q = question.lower()
    extracted = extract_symptoms_llm(question)
    if not extracted:
        return None

    crop_boost = {"tomato": 3.0, "potato": 3.0, "pepper": 3.0}
    detected_crop = None

    for crop in crop_boost:
        if crop in q:
            detected_crop = crop
            break

    scores = {}

    for disease, info in KNOWLEDGE["Diseases"].items():
        symptom_text = info.get("Symptoms", "").lower()
        score = 0

        for token in extracted:
            if token in symptom_text:
                score += 1.5

        if detected_crop:
            if detected_crop in disease.lower():
                score *= crop_boost[detected_crop]
            else:
                score *= 0.2

        scores[disease] = score

    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else None


# ----------------------------------------------------
# SAFE INTENT PARSER (No missing keys ever)
# ----------------------------------------------------
def interpret_question_with_llm(question: str):
    prompt = f"""
Extract agricultural intent as JSON.

Fields:
crop: tomato/potato/pepper/none
disease: name or none
symptoms: list
intent: fertilizer/cure/identify/prevention/climate/deficiency/pest/general
fertilizer_requested: yes/no

Question: "{question}"
Return JSON only.
"""

    raw = ask_offline_llm(prompt)

    safe = {
        "crop": "none",
        "disease": "none",
        "symptoms": [],
        "intent": "general",
        "fertilizer_requested": "no"
    }

    try:
        data = json.loads(raw)

        for k in safe:
            if k not in data:
                data[k] = safe[k]

        q = question.lower()

        if "fertilizer" in q:
            data["fertilizer_requested"] = "yes"

        if "tomato" in q:
            data["crop"] = "tomato"
        elif "potato" in q:
            data["crop"] = "potato"
        elif "pepper" in q:
            data["crop"] = "pepper"

        return data

    except:
        q = question.lower()

        if "fertilizer" in q:
            safe["fertilizer_requested"] = "yes"

        if "tomato" in q:
            safe["crop"] = "tomato"
        elif "potato" in q:
            safe["crop"] = "potato"
        elif "pepper" in q:
            safe["crop"] = "pepper"

        return safe


# ----------------------------------------------------
# MASTER ROUTER (Final — crop-aware disease matching)
# ----------------------------------------------------
def auto_route(question: str, image_path: str = None, use_llm=True):
    q = question.lower()

    # 1) Image → CNN override
    if image_path:
        pred, result = system.diagnose_and_advise(image_path, question, ask_llm=use_llm)
        return {"module": "Disease Diagnosis", "prediction": pred, "result": result}

    # 2) LLM intent parse
    intent = interpret_question_with_llm(question)
    crop = intent.get("crop", "none")
    explicit_disease = intent.get("disease", "none")
    main_intent = intent.get("intent", "general")
    fert_req = intent.get("fertilizer_requested", "no")

    # 3) Explicit disease detected by LLM -> PREFERRING crop-aware match
    if explicit_disease and explicit_disease != "none":
        # prefer disease that also mentions crop if crop is known
        if crop != "none":
            # search for disease with explicit_disease substring + crop
            for dname in KNOWLEDGE["Diseases"]:
                if explicit_disease.lower() in dname.lower() and crop in dname.lower():
                    return {"module": "Disease Information", "disease": dname, "data": KNOWLEDGE["Diseases"][dname]}
        # fallback: generic match
        for dname in KNOWLEDGE["Diseases"]:
            if explicit_disease.lower() in dname.lower():
                return {"module": "Disease Information", "disease": dname, "data": KNOWLEDGE["Diseases"][dname]}

    # 4) Pest detection (high priority)
    pest_keywords = ["mite", "aphid", "whitefly", "worm", "thrip", "web", "speckle"]
    if any(w in q for w in pest_keywords) or main_intent == "pest":
        # If crop known, check crop diseases first
        if crop != "none":
            for dname, info in KNOWLEDGE["Diseases"].items():
                if crop in dname.lower() and ("mite" in info.get("Symptoms","").lower() or "mite" in info.get("Cause","").lower()):
                    return {"module": "Disease Info (Symptoms)", "disease": dname, "data": info}
        # otherwise return pest module
        return {"module": "Pest", "data": KNOWLEDGE["Pests"]}

    # 5) DEFICIENCY detection (do this BEFORE disease inference)
    deficiency_keywords = ["yellow", "chlorosis", "purple", "purplish", "interveinal", "veins", "pale"]
    if any(w in q for w in deficiency_keywords) or main_intent == "deficiency":
        return {"module": "Deficiency", "data": KNOWLEDGE["Deficiencies"]}

    # 6) Symptom-based disease inference (ML + keyword)
    ml_guess = ml_style_symptom_match(question)
    kw_guess = match_disease_by_symptoms(question)

    # prefer crop-aware guesses: if crop present, prefer disease matching crop
    def prefer_crop_candidate(candidate):
        if not candidate:
            return None
        if crop != "none" and crop in candidate.lower():
            return candidate
        # if not matching crop, still ok as fallback
        return candidate

    final_guess = None
    # try ML guess first but with crop preference
    if ml_guess:
        final_guess = prefer_crop_candidate(ml_guess) or ml_guess
    if not final_guess and kw_guess:
        final_guess = prefer_crop_candidate(kw_guess) or kw_guess

    if final_guess:
        return {"module": "Disease Info (Symptoms)", "disease": final_guess, "data": KNOWLEDGE["Diseases"][final_guess]}

    # 7) Fertilizer logic (strict matching + crop-safe)
    if fert_req == "yes":
        # 7a: check explicit disease tokens (strict) with crop preference
        # Make a list of disease-specific tokens from disease names
        for dname, info in KNOWLEDGE["Diseases"].items():
            d_lower = dname.lower()
            # skip if crop known but disease not of that crop
            if crop != "none" and crop not in d_lower:
                continue
            # exact token match: check if both crop and disease token present when crop is in question
            tokens = [t for t in ["early blight", "late blight", "bacterial spot", "leaf mold", "septoria", "spider mites"]]
            for token in tokens:
                if token in q and token in d_lower:
                    return {"module": "Disease-Specific Fertilizer", "disease": dname, "fertilizer": info.get("Fertilizer Recommendation", "Not specified")}

        # 7b: if crop present, return crop-level fertilizers
        if crop != "none":
            ferts = {d: KNOWLEDGE["Diseases"][d].get("Fertilizer Recommendation", "Not specified")
                     for d in KNOWLEDGE["Diseases"] if crop in d.lower()}
            if ferts:
                return {"module": f"{crop.capitalize()} Fertilizer", "data": ferts}

        # 7c: global fallback: show dataset-wide fertilizer table
        ferts_all = {d: KNOWLEDGE["Diseases"][d].get("Fertilizer Recommendation", "Not specified") for d in KNOWLEDGE["Diseases"]}
        return {"module": "Fertilizer", "data": ferts_all}

    # 8) Climate questions
    if main_intent == "climate":
        return {"module": "Climate", "data": KNOWLEDGE["Climate"]}

    # 9) LLM grounded fallback
    if use_llm:
        prompt = ("Use ONLY this dictionary:\n" + json.dumps(KNOWLEDGE, indent=2) +
                  f"\n\nQuestion: {question}\nDo NOT hallucinate.")
        return {"module": "General", "answer": ask_offline_llm(prompt)}

    # 10) absolute fallback
    return {"module": "General", "answer": "No matching module found."}


# -------------------------
# Quick smoke tests (uncomment to run)
# -------------------------
# print(auto_route("Which fertilizer for tomato early blight?"))
# print(auto_route("My tomato leaves have brown rings, what is the issue?"))
# print(auto_route("There are tiny webs and yellow speckles on my tomato leaves"))
# print(auto_route("Leaves are turning purple, what deficiency is this?"))


In [202]:
# TEST
auto_route("Which fertilizer for tomato early blight?")


{'module': 'Disease Information',
 'disease': 'Tomato Early Blight',
 'data': {'Symptoms': 'Concentric brown rings, leaf yellowing.',
  'Cause': 'Alternaria solani.',
  'Organic Control': 'Compost tea, neem oil.',
  'Chemical Control': 'Mancozeb, chlorothalonil.',
  'Prevention': 'Remove infected leaves.',
  'Fertilizer Recommendation': 'NPK 10-20-20.'}}

In [203]:
auto_route("My tomato leaves have brown rings, what is the issue?")


{'module': 'Disease Info (Symptoms)',
 'disease': 'Tomato Early Blight',
 'data': {'Symptoms': 'Concentric brown rings, leaf yellowing.',
  'Cause': 'Alternaria solani.',
  'Organic Control': 'Compost tea, neem oil.',
  'Chemical Control': 'Mancozeb, chlorothalonil.',
  'Prevention': 'Remove infected leaves.',
  'Fertilizer Recommendation': 'NPK 10-20-20.'}}

In [204]:
auto_route("There are tiny webs and yellow speckles on my tomato leaves")


{'module': 'Disease Info (Symptoms)',
 'disease': 'Tomato Spider Mites',
 'data': {'Symptoms': 'Leaf bronzing, tiny webs, yellow speckles.',
  'Cause': 'Mites.',
  'Organic Control': 'Neem oil (2 ml/L), soap spray.',
  'Prevention': 'Increase humidity.'}}

In [205]:
auto_route("Leaves are turning purple, what deficiency is this?")


{'module': 'Deficiency',
 'data': {'Nitrogen Deficiency': {'Symptoms': 'Pale yellow older leaves.',
   'Fix': 'Compost, cow manure, urea (1 tsp/L).',
   'Prevention': 'Regular compost application.'},
  'Phosphorus Deficiency': {'Symptoms': 'Purplish leaves, slow growth.',
   'Fix': 'DAP, bone meal.',
   'Prevention': 'Maintain soil pH 6–7.'},
  'Potassium Deficiency': {'Symptoms': 'Brown leaf edges (scorching).',
   'Fix': 'MOP fertilizer.',
   'Prevention': 'Balanced NPK schedule.'},
  'Calcium Deficiency': {'Symptoms': 'Blossom end rot.',
   'Fix': 'Calcium nitrate (1 tsp/L).',
   'Prevention': 'Avoid irregular watering.'},
  'Magnesium Deficiency': {'Symptoms': 'Yellowing between veins on older leaves.',
   'Fix': 'Epsom salt spray (1 tsp/L).',
   'Prevention': 'Maintain soil pH 6–7.'},
  'Iron Deficiency': {'Symptoms': 'Yellowing between veins on young leaves.',
   'Fix': 'Ferrous sulfate spray.',
   'Prevention': 'Avoid alkaline soil.'}}}